In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog

## Markov Decision Process (MDP)

An MDP is defined by the tuple $\langle \mathcal{S}, \mathcal{A}, \mathcal{T}, \mathcal{R}, \gamma  \rangle$ where,

$\mathcal{S}$ is the state space

$\mathcal{A}$ is the action space

$\mathcal{T}$ is the transition model

$\mathcal{R}$ is the reward model

$\gamma$ is the discount factor


## State Space

In [2]:
TIME_HORIZON = 10
num_damage_states = 5

num_states = TIME_HORIZON * num_damage_states

state_space = {}
key = 1

for time in range(1, TIME_HORIZON+1):
    for state in range(1, num_damage_states+1):
        state_space[key] = (time, state)
        key += 1

print(state_space)

{1: (1, 1), 2: (1, 2), 3: (1, 3), 4: (1, 4), 5: (1, 5), 6: (2, 1), 7: (2, 2), 8: (2, 3), 9: (2, 4), 10: (2, 5), 11: (3, 1), 12: (3, 2), 13: (3, 3), 14: (3, 4), 15: (3, 5), 16: (4, 1), 17: (4, 2), 18: (4, 3), 19: (4, 4), 20: (4, 5), 21: (5, 1), 22: (5, 2), 23: (5, 3), 24: (5, 4), 25: (5, 5), 26: (6, 1), 27: (6, 2), 28: (6, 3), 29: (6, 4), 30: (6, 5), 31: (7, 1), 32: (7, 2), 33: (7, 3), 34: (7, 4), 35: (7, 5), 36: (8, 1), 37: (8, 2), 38: (8, 3), 39: (8, 4), 40: (8, 5), 41: (9, 1), 42: (9, 2), 43: (9, 3), 44: (9, 4), 45: (9, 5), 46: (10, 1), 47: (10, 2), 48: (10, 3), 49: (10, 4), 50: (10, 5)}


## Action Space

The action space has 3 actions and we carry out these actions at the beginning of the time-step

0: Do nothing - component undergoes deterioration due to the environment

1: Repair - component moves back by 1 damage state (+undergoes deterioration due to the environment)

2: Replace - component is replaced (+undergoes deterioration due to the environment)

In [3]:
DO_NOTHING = 0
MINOR_REPAIR = 1
REPLACE = 2

action_space = [DO_NOTHING, MINOR_REPAIR, REPLACE]

num_actions = len(action_space)

## Transition Model

In [4]:
TRANSITION_MODEL = np.array([[0.7, 0.3, 0.0, 0.0, 0.0],
                             [0.0, 0.6, 0.4, 0.0, 0.0],
                             [0.0, 0.0, 0.5, 0.5, 0.0],
                             [0.0, 0.0, 0.0, 0.2, 0.8],
                             [0.0, 0.0, 0.0, 0.0, 1.0]])

## Reward model

In [5]:
REPAIR_COST = -25
REPLACE_COST = -50

REWARDS = [0, REPAIR_COST, REPLACE_COST]

PENALTY = -500

## Discount Factor

In [6]:
DISCOUNT_FACTOR = 0.9

In [7]:
def MDP_model(current_state, action):

    """_summary_

    Returns
    -------
    current_state: 
        _description_

    action:
        
    """    

    current_time, current_damage_state = current_state
    next_time = current_time + 1
    
    output = []

    # action = 'do-nothing'
    # damage state does not change

    # action = 'minor-repair'
    if action == 1:
        # move back by one state
        # but not lower than 1
        # but no minor repair for failure
        if current_damage_state != 5:
            current_damage_state = max(1, current_damage_state-1)

    # action = 'replace'
    elif action == 2:
        # replacing leads to initial undamaged state
        current_damage_state = 1

    for next_damage_state in range(1, num_damage_states+1):

        next_state = (next_time, next_damage_state)
        prob = TRANSITION_MODEL[current_damage_state-1, next_damage_state-1]
        reward = REWARDS[action]

        if next_damage_state == 5:
            reward += PENALTY

        output.append((prob, next_state, reward))

    return output

In [8]:
MDP_model(state_space[5], 2)

[(0.7, (2, 1), -50),
 (0.3, (2, 2), -50),
 (0.0, (2, 3), -50),
 (0.0, (2, 4), -50),
 (0.0, (2, 5), -550)]

## Linear Programming (Primal formulation)



Linear programming: minimize a linear objective function subject to linear equality and inequality constraints.
Linear programming solves problems of the following form:

$
\begin{aligned} 
\underset{v \in \mathbb{R}^{|\mathcal{S}|}}{\operatorname{minimize}} & \quad \sum_{ s} c(s) v(s) \\ 
\text { subject to } & \quad v(s) \geq  \sum_{s^{\prime}} P\left(s^{\prime} \mid s, a\right) \Big [ r(s, a, s') + \gamma \cdot  v\left(s^{\prime}\right)\Big ], \quad \forall a \in \mathcal{A}, s \in \mathcal{S} \\ & \quad v(s) \text { unconstrained }, \forall s \in \mathcal{S}
\end{aligned}
$

The optimization problem is an LP with $|\mathcal{S}|$ variables and $|\mathcal{S}| \times|\mathcal{A}|$ constraints. We rewrite the above LP in the standard formulation as:

$\begin{aligned} \underset{v \in \mathbb{R}^{|\mathcal{S}|}}{\operatorname{minimize}} & \quad \sum_{s} c(s) v(s) \\ 
\text { subject to } & \quad - v(s) + \gamma \cdot \sum_{s^{\prime}}  P\left(s^{\prime} \mid s, a\right) \cdot  v\left(s^{\prime}\right)  \leq - \sum_{s^{\prime}} P\left(s^{\prime} \mid s, a\right) \cdot r(s, a, s'), \quad \forall a \in \mathcal{A}, s \in \mathcal{S} \\ & \quad v(s) \text { unconstrained }, \forall s \in \mathcal{S}\end{aligned}$

([Adapted from "Linear Programming in Reinforcement learning"](https://escholarship.mcgill.ca/downloads/xs55mh725))

We use the [scipy.optimize.linprog](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html) module to solve the problem and use the same variable notation for convinience.

$
\begin{aligned}
\underset{v}{\operatorname{minimize}} \quad & c^T v \\
\text { such that } & A_{u b} v \leq b_{u b}, \\
& A_{e q} v=b_{e q}, \\
& l \leq v \leq u,
\end{aligned}
$

where $v$ is a vector of value of states; $c, b_{u b}, b_{e q}, l$, and $u$ are vectors; and $A_{u b}$ and $A_{e q}$ are matrices.

### Inequality constraints

In [18]:
# we exclude the terminal states here and include them in equality constraints next
num_constraints = (num_states-num_damage_states) * num_actions

# mu(s) in the above formulation
c = np.ones(num_states)

# Matrix corresponding to the inequality constraints
A_ub = np.zeros((num_constraints, num_states))

# Vector corresponding to the inequality constraints
b_ub = np.zeros(num_constraints)

k = 0 # counter for counting constraint

# loop over all states except terminal states
for idx_state in range(num_states-num_damage_states):
    # loop over all actions in action space
    for action in action_space:
        
        A_ub[k, idx_state] = -1

        state = state_space[idx_state+1]

        # enumerate all (one-step) future states given a (state, action) pair
        for prob, next_state, reward in MDP_model(state, action):
            
            # compute global index of state 
            idx_next_state = (next_state[0]-1) * num_damage_states + next_state[1]-1

            # add entries in corresponding next states
            A_ub[k, idx_next_state] = DISCOUNT_FACTOR * prob 
            b_ub[k] += - prob * reward

        k += 1

### Equality constraints

In [19]:
# equality constraints for terminal states
A_eq = np.zeros((num_damage_states, num_states))

# set coefficient of terminal states to 1
for idx_state in range(num_damage_states):
    A_eq[idx_state, -1 - idx_state] = 1

# value of terminal states is 0
b_eq = np.zeros(num_damage_states)

In [20]:
# NOTE: we must specify bounds for the unconstrained LP since default is (0, None) (non-negative)
result = linprog(c, 
                 A_ub=A_ub, 
                 b_ub=b_ub, 
                 A_eq=A_eq, 
                 b_eq=b_eq, 
                 bounds=(-math.inf, math.inf))

print(f"Has Linear Program converged?: {result.success}")

Has Linear Program converged?: True


In [48]:
print(f"Value of states: [time_horizon, num_states] \n {np.around(result.x.reshape(TIME_HORIZON, num_damage_states),3)}")

Value of states: [time_horizon, num_states] 
 [[-16.602 -33.016 -49.573 -66.602 -66.602]
 [-13.485 -30.024 -46.677 -63.485 -63.485]
 [-10.054 -26.484 -43.672 -60.054 -60.054]
 [ -6.519 -22.024 -40.531 -56.519 -56.519]
 [ -3.357 -16.312 -36.711 -53.357 -53.357]
 [ -1.094  -9.882 -30.488 -51.094 -51.094]
 [ -0.     -4.05  -21.375 -46.375 -50.   ]
 [ -0.     -0.    -11.25  -36.25  -50.   ]
 [ -0.     -0.     -0.    -25.    -50.   ]
 [ -0.     -0.     -0.     -0.     -0.   ]]


## Optimal policy

The constraint corresponding to the best action will be "tight" or alternatively, will have "no slack"

In [36]:
# get slack of the inequality constraints
optimal_policy = result.slack.reshape((num_states-num_damage_states),  num_actions)

# arguments of actions having no (minimum) slack
optimal_policy = np.argmin(optimal_policy, axis=1)

In [39]:
# optimal policy (excluding terminal states)
optimal_policy.reshape(TIME_HORIZON-1, num_damage_states)

array([[0, 0, 0, 2, 2],
       [0, 0, 0, 2, 2],
       [0, 0, 0, 2, 2],
       [0, 0, 0, 2, 2],
       [0, 0, 0, 2, 2],
       [0, 0, 0, 2, 2],
       [0, 0, 0, 1, 2],
       [0, 0, 0, 1, 2],
       [0, 0, 0, 1, 2]])